## Second kind of the task of querying : What is the relationship between XXX and YYY?
BRICK provides a streamlined approach for querying relationships between two entities.
For example, the user need to query the relationship between Isl1 and PP cell, use BRICK module to query the relationship between Isl1 and PP cell.

In [1]:
user_question = "What is the relationship between Isl1 and pp Cells?"

Load the BRICK module.

In [2]:
import sys
import BRICK
import scanpy as sc

from dotenv import dotenv_values
import json
config = dotenv_values('/workspace/data/brick.env')
kg_url = config.get('KG_URL')
kg_auth = (config.get('KG_AUTH_USER'), config.get('KG_AUTH_PASS'))
model = config.get("MODEL_TYPE")
url = config.get("LLM_URL")
api_key = config.get("API_KEY")
llm_params = json.loads(config.get("LLM_PARAMS"))

BRICK.config(url=kg_url, auth=kg_auth)
BRICK.config_llm(modeltype=model, base_url=url,api_key=api_key, llm_params=llm_params)

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Graph database has been configured and initialized successfully.
LLM has been configured and initialized successfully.


Use BRICK.qr.query_relation to query the relationship between Isl1 and PP cell.
The function need the type of source entity and target entity.Remember the source entity and target entity need to be in the format of a list, set or tuple.

In [3]:
source_entity_set, target_entity_set = BRICK.inp.extract_entities_relation(user_question)
print(source_entity_set, target_entity_set)

['Isl1'] ['pp Cells']


In [4]:
client = BRICK.se.BRICKSearchClient()
print(f"字符串索引: {client.search_config.string_index_name}")
print(f"向量索引: {client.search_config.vector_index_name or '已禁用'}\n")

payload_fuzzy_auto = client.build_untyped_payload(query_text=source_entity_set, top_k=1)
print(f">>> [Fuzzy] 未指定类别：全类别检索（query_text='{source_entity_set}'）")
df_all_s = client.search_fuzzy(payload_fuzzy_auto)
df_all_s

字符串索引: brick_index_*_string
向量索引: brick_index_*_vector

>>> [Fuzzy] 未指定类别：全类别检索（query_text='['Isl1']'）


,entity_id,primary_name,type_key,node_type,string_score,matched_alias
0,NCBI:16392,Isl1,Gene|Protein,Gene,67.38008,None


In [5]:
payload_fuzzy_auto = client.build_untyped_payload(query_text=target_entity_set, top_k=1)
print(f">>> [Fuzzy] 未指定类别：全类别检索（query_text='{target_entity_set}'）")
df_all_t = client.search_fuzzy(payload_fuzzy_auto)
df_all_t

>>> [Fuzzy] 未指定类别：全类别检索（query_text='['pp Cells']'）


,entity_id,primary_name,type_key,node_type,string_score,matched_alias
0,NCBI:104145,Pp,Gene|Protein,Gene,55.151295,None


In [6]:
sentity_name = df_all_s.head(1)['primary_name'].iloc[0]
tentity_name = df_all_t.head(1)['primary_name'].iloc[0]

In [7]:
relation_frame = BRICK.qr.query_relation(source_entity_set=sentity_name, source_entity_type="Gene", target_entity_set=tentity_name, target_entity_type="Cell")

Then use BRICK.inp.interpret_query to get the answer.


In [8]:
ans = BRICK.inp.interpret_query(user_question,relation_frame)
print(ans)

Given the provided table is empty (it has no columns and no data), there's no information available to answer the question "What is the relationship between Isl1 and pp Cells?" directly from the table.

However, I can provide some general knowledge about the relationship between Isl1 and pancreatic progenitor (pp) cells:

- **Isl1 (Islet-1)**: This is a LIM homeodomain transcription factor that plays a crucial role in the development of various tissues, including the pancreas. It is involved in the specification and differentiation of pancreatic endocrine cells.
  
- **Pancreatic Progenitor (pp) Cells**: These are undifferentiated cells in the developing pancreas that have the potential to differentiate into various cell types, including endocrine, exocrine, and ductal cells.

- **Relationship**: Isl1 is known to be expressed in a subset of pancreatic progenitor cells and is important for the specification and differentiation of these cells into specific lineages, particularly endocrin

Besides, users might also want to query the relationship between three or more entities.

In [9]:
user_question2 = "What is the relationship between Isl1 ,PP cell, and GIP cell?"

The meaning of the query is to find the relationship between Isl1 and PP cell, Isl1 and GIP cell, and PP cell and GIP cell.
So we need to query the relationship between Isl1 and PP cell, Isl1 and GIP cell, and PP cell and GIP cell.

In [10]:
relations = BRICK.qr.query_relation(source_entity_set=["Isl1"],source_entity_type="Gene", target_entity_set=["PP cell", "GIP cell"], target_entity_type="Cell")
relations

,path.0.def,path.0.id,path.0.name,path.0.synonym,path.0.type,path.1,path.1.condition,path.1.info_source,path.1.info_source_length,path.1.original_relation,path.1.relation,path.1.relation_confidence,path.2.def,path.2.id,path.2.name,path.2.synonym,path.2.type
0,"ISL1 transcription factor, LIM/homeodomain<loc>:13 D2.2|13 64.87 cM<xref>MGI:101791|ENSEMBL:ENSMUSG00000042258</xref>",NCBI:16392,Isl1,Undef,Gene,produce,"[UBERON:0001264, Undef]","[CELL_TAXONOMY:PANGLAODB, CELL_TAXONOMY:ONCLASS:PMID:34548483]",2,"[produce, produce]",produce,NaN,A cell that stores and secretes pancreatic polypeptide hormone.<xref>FMA:62938|FMA:83409</xref>,CL:0000696,PP cell,type F enteroendocrine cell,Cell
1,"ISL1 transcription factor, LIM/homeodomain<loc>:13 D2.2|13 64.87 cM<xref>MGI:101791|ENSEMBL:ENSMUSG00000042258</xref>",NCBI:16392,Isl1,Undef,Gene,marker_of,"[Undef, UBERON:0001264]","[PanglaoDB, Cell_Taxonomy:PanglaoDB, Cell_Taxonomy:OnClass:PMID:34548483]",3,"[marker_of, marker_of, marker_of]",marker_of,"[1, 1, 1]",A cell that stores and secretes pancreatic polypeptide hormone.<xref>FMA:62938|FMA:83409</xref>,CL:0000696,PP cell,type F enteroendocrine cell,Cell
2,"ISL1 transcription factor, LIM/homeodomain<loc>:13 D2.2|13 64.87 cM<xref>MGI:101791|ENSEMBL:ENSMUSG00000042258</xref>",NCBI:16392,Isl1,Undef,Gene,produce,[Undef],[CELL_TAXONOMY:ONCLASS:PMID:34548483],1,[produce],produce,NaN,An enteroendocrine cell of duodenum and jejunum that produces gastric inhibitory peptide.<xref>FMA:62942</xref>,CL:0002278,GIP cell,type K enteroendocrine cell,Cell
3,"ISL1 transcription factor, LIM/homeodomain<loc>:13 D2.2|13 64.87 cM<xref>MGI:101791|ENSEMBL:ENSMUSG00000042258</xref>",NCBI:16392,Isl1,Undef,Gene,marker_of,[Undef],[Cell_Taxonomy:OnClass:PMID:34548483],1,[marker_of],marker_of,[1],An enteroendocrine cell of duodenum and jejunum that produces gastric inhibitory peptide.<xref>FMA:62942</xref>,CL:0002278,GIP cell,type K enteroendocrine cell,Cell


We get one relations frame, then we need to interpret them.

In [11]:
ans = BRICK.inp.interpret_query(user_question2,relations)
print(ans)

Based on the provided table, we can analyze the relationships between Isl1, PP cells, and GIP cells.

### What is the relationship between Isl1, PP cell, and GIP cell?

- **Isl1 (ISL1 transcription factor, LIM/homeodomain)**:
  - **Gene**: Isl1 is a gene that encodes the ISL1 transcription factor, which is involved in the development and function of various cell types.
  
- **PP Cell (Pancreatic Polypeptide Cell)**:
  - **Cell Type**: PP cells are a type of enteroendocrine cell found in the pancreas. They store and secrete pancreatic polypeptide hormone.
  - **Relationship with Isl1**:
    - The table indicates that Isl1 is a "marker_of" PP cells. This means that Isl1 is expressed in PP cells and can be used as a marker to identify these cells.
    - Additionally, Isl1 is also listed as "produce" in relation to PP cells, suggesting that Isl1 may play a role in the production or function of PP cells.

- **GIP Cell (Gastric Inhibitory Peptide Cell)**:
  - **Cell Type**: GIP cells are ano